# Getting Started with the Binary Bosonic Solver


## Introduction

A photonic quantum processor can be used to solve combinatorial optimisation problems using ORCA's binary bosonic solver (BBS) algorithm. This is a type of variational quantum algorithm, in which the distribution of photons at the output of a PT Series device is mapped to a binary string, a *cost function* is calculated for these strings, and a classical optimisation loop is used to find the beam splitter parameters in the PT Series that generate a state that minimises the cost function. Here, we assume that non-number resolving detectors are used. In other words, if there is more than one photon in a mode at the output, this just returns a value '1'. We then use parametrisable bit flips which flip each of the output bits from this distribution with a given probability. These probabilities, and the angles in the beamsplitter, are trainable parameters. Over the course of the algorithm we train them to give better results.

<center><img src="figures/binary_bosonic_solver_scheme.png" alt="Hybrid GAN" width="700"/></center>
<center>Figure 1: Overview of ORCA's algorithm for solving binary optimisation problems.</center>

In this notebook, we will cover the following:
- Detailed setup of the BBS algorithm to solve a simple optimisation task
- Using a `logger` object to store training data
- Setup of a *knapsack problem* with constraints and solving it with BBS

The following code block does all of the relevant imports:

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

if os.getcwd().endswith("notebooks"):
    os.chdir("..")

from ptseries.algorithms.binary_solvers import BinaryBosonicSolver
from ptseries.common.logger import Logger

## Setting Up the Algorithm

In the following section we will go through a simple example of an optimisation problem, where we will show how to set up the BBS algorithm to solve a simple problem. We will first consider a problem where it is simple to figure out the optimal solution. We're going to look at a solution space of length $6$ binary strings. We set a binary string $s$ which we will call the 'solution vector' and then we'll task ourselves with finding the length $6$ binary string $v$ that minimises the value: $$v \cdot \hat{s} - v \cdot s $$
where $\hat{s}$ is the vector $s$ with all of it's bits flipped (i.e. 1s sent to 0s and vice versa). The '$\cdot$' here represents the *dot product* between two vectors. In this case, the solution (the $v$ which minimises this value) will be exactly equal to $s$. We will go through how to set up the BBS algorithm for this problem.

We first define our solution_vector $s$ and the problem size (length of bit strings we consider). Then, we set the *cost function* which calculates the value we're trying to minimise, taking as input a binary string and outputting a calculation of the formula we defined above.

In [ ]:
# Defines the solution vector s
solution_vector = np.array([1, 0, 1, 0, 1, 0, 1, 0])

# Calculates the length of bit strings we will be considering for solutions
problem_size = len(solution_vector)


# Defines our cost function. We take a bit string as input, and then output the value of the cost function calculated at that bit string.
# In this case, this is taking the dot product formula defined above.
def cost_fn(bit_str):
    # Stores the value of s^ : the solution vector s with all of the bits flipped
    flipped_solution_vector = ~solution_vector % 2

    # Returns the cost function defined above calculated with 'bit_str' in place of v and 'solution_vector' in place of s
    return -np.dot(solution_vector, bit_str) + np.dot(flipped_solution_vector, bit_str)

In the following code block, we will define a `logger` object from the SDK which keeps track of the training data from the BBS algorithm. In particular, the algorithm is trying to minimise a 'loss function' that defines how good the solutions considered are. Here, the loss function is equal to the averages of the cost functions calculated for all of the output bit strings during a single iteration of the algorithm. The logger keeps track of these values after each update step.

We next instantiate the `BinaryBosonicSolver` object as the variable `bbs`. When doing this, we need to tell the algorithm the `problem size` (length of bit strings we consider, labelled `pb_dim` i.e. problem dimension), the cost function we're using (labelled `objective`) and also the parameters of the time bin interferometer that we'll be using (`tbi_params`). In this case, we'll set our time bin interferometer to have a single loop.

In [ ]:
logger = Logger(log_dir=None)

bbs = BinaryBosonicSolver(
    pb_dim=problem_size, objective=cost_fn, tbi_params={"tbi_type": "single-loop"}
)

Next, we will call the `.train` method of the bbs algorithm. We set the following parameters:
- `learning_rate`: Defines the scaling of the update steps we take for the parametrisable angles inside the tbi
- `learning_rate_flip`: Defines the scaling of the update steps we take for the parametrisable bit flips
- `updates`: Number of iterations we do of the algorithm, after each one updating the parameters
- `print_frequency`: Frequency with which we print out the average cost function values the algorithm is achieving
- `logger`: Defines the logger we'll be using

In [ ]:
bbs.train(
    learning_rate=5e-2,
    learning_rate_flip=1e-1,
    updates=200,
    print_frequency=40,
    logger=logger,
)

At the end of this training cycle, we see that the loss being achieved has gone down, indicating that the bit strings being produced by our algorithm are closer to the true solution. Now, we output the best solution found by the algorithm, meaning the bit string encountered which had the lowest cost function. We use the `config_min_encountered` member to do this.

In [ ]:
best_sol = bbs.config_min_encountered

print(f"Best solution found is {best_sol} with cost function value {cost_fn(best_sol)}")

We see that the solution the BBS algorithm found is exactly equal to the solution vector that we defined initially. In other words, the algorithm has found the solution. Note also that the loss the algorithm was achieving near the end of training was very close to the true minimal value of -4. This means that the algorithm, near the end of training, had most of its binary string outputs equal to the true solution.

### (Aside) Using the Logger

The `logger` stored the data about all of the average cost function values we observed during training. We can then use it to nicely plot these values to investigate the performance of the algorithm.

In [ ]:
# Gets the number of updates as a list of values we will use to plot the x axis
x_values = list(map(int, logger.logs["energy_avg"].keys()))

# Gets the average cost function value calculated at each update step
cost_fn_avg = list(logger.logs["energy_avg"].values())

plt.plot(x_values, cost_fn_avg)

plt.xlabel("Update Number")
plt.ylabel("Average Cost Function Value")
plt.show()


In fact, the logger stores even more information. The logger has kept track of the highest and lowest cost function values encountered at each update step.

## Setting up the Algorithm - A Simple Knapsack Problem
Here we will now look at a slightly more complicated optimisation task, and reviewing how we set up the BBS algorithm to find good solutions. In this example, we will consider a simple instance of a *knapsack problem*. Here, the 0 and 1 values in a bit string represent whether we take an item or not. Each item gives us value, but also adds weight to the knapsack. We want to choose which items to take to get the most value, but also not go over the amount of weight we can carry in the knapsack.

We will consider problems where we have $m$ items we could take, each having a weight $w_i$ and value $v_i$. Our knapsack has a maximum weight capacity of $M$. We want to maximise the sum of the values we get from the items we take, denoted by $V$. To define possible solutions for the problem, we'll use length $m$ bit strings $x_0 x_1 ... x_{m-1}$ where, if $x_i = 1$, it means that we're taking item $i$. We can formulate this problem precisely as wanting to find the bit string giving us:
$$\max V = \sum_i v_i x_i  \quad  \text{s.t.} \quad \sum_i w_i x_i \le M$$

We'll now consider an exact problem, where we'll set $m = 7$, giving us the number of items (i.e. length of bit strings) that we'll consider, and we'll set some $v_i$ and $w_i$ values.

In [ ]:
# Sets the max weight capacity for the knapsack
M = 20

# Defines the size of the problem
problem_size = 7

# Sets the values and weights for each item
values = np.array([5, 7, 1, 10, 9, 12, 13], dtype=int)

weights = np.array([10, 5, 2, 6, 8, 7, 4], dtype=int)

Unlike the previous example, we can't just make our cost function equal to the value of items a bit string takes, as then the best solution would clearly be to just take every single item. We need to dissuade the algorithm from considering solutions that break the weight constraint. We will do this by adding a penalty term to the cost function, that adds a big penalty if a bit string encodes a solution going over the weight limit. 

This time, we also need to feed in the values, weights and weight capacity we defined into the function. So, we will create a function with two layers. The first layer `cost_fn_gen` will create the cost function we want with the specified values. This way, we can easily generate the same cost function for new item values and weights. It's important to note that here, we calculate the value associated with taking certain items but then the cost function returns the negative of this value. This is because, for the knapsack problem, we want to get the most valuable choice of items but the BBS algorithm is trying to minimise the cost function. Thus, we need to take the negative of the value_total at the end for the algorithm to find solutions maximising the value total.

In [ ]:
# Generates a cost function with the desired parameters
def cost_fn_gen(values, weights, weight_limit, penalty):
    def cost_fn(bit_str):
        # Calculates the total value associated with taken items
        value_total = np.dot(values, bit_str)

        # Calculates the total weight associated with taken items
        weight_total = np.dot(weights, bit_str)

        # If the solution being considered goes over the weight limit, we add a big penalty term
        if weight_total > weight_limit:
            value_total += -penalty

        # Returns the negative of the value total
        return (-1) * value_total

    # Returns the cost function we've generated
    return cost_fn

Now, using this generator, we'll create the cost function that we want, and we will take a penalty term of 50 meaning that any solution which goes over the weight limit will be heavily penalised. Thus, the BBS algorithm will steer away from such solutions as they will have a low cost function value. Then, we set up the BBS algorithm and train it exactly as we did in the first example. This time, we do 400 update steps.

In [ ]:
cost_fn = cost_fn_gen(values=values, weights=weights, weight_limit=M, penalty=50)

bbs = BinaryBosonicSolver(
    pb_dim=problem_size, objective=cost_fn, tbi_params={"tbi_type": "single-loop"}
)

bbs.train(learning_rate=5e-2, learning_rate_flip=1e-1, updates=400, print_frequency=50)


Again, we output the best solution found, with the exact same code as in the first example except we take the negative of the cost function to make up for the fact that we were taking the negative of the value earlier. We also output the weight of the solution we've found.

In [ ]:
best_sol = bbs.config_min_encountered

print(
    f"Best solution found is {best_sol} with a value of {(-1) * cost_fn(best_sol)}, while having weight {np.dot(weights, best_sol)}"
)

And there we have it! We found a solution with value 36 and weight 19. In fact, if you go through the optimisation problem by hand you can see that this is the exact solution to the problem.

## Next Steps
In this notebook we've gone through two examples of how to setup the BBS algorithm to solve binary optimisation tasks which have a non-QUBO formulation. There are three more advanced notebooks about the BBS algorithm, found within the `tutorial_notebooks/optimisation_notebooks` folder each of which looks at a different example optimisation task in depth.
- `QUBO_&_max_cut.ipynb`: Introduces QUBO formulated problems, and goes over solving a max-cut problem in QUBO formulation
- `route_optimisation.ipynb`: Introduces a travelling salesman task, and how to solve it using the BBS algorithm
- `workshop_optimisation.ipynb`: Considers a larger scale knapsack problem to the one seen above